In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

In C:\Users\Ayush\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Ayush\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Ayush\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Ayush\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Ayush\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [3]:
data = pd.read_csv("All_States_Daily_Count.csv",index_col=0)
data.iloc[-1:, -4:] = data.iloc[1:-1,-4:].astype(int).sum().to_list()

In [4]:
total_cases = pd.DataFrame()
total_cured = pd.DataFrame()
total_deaths = pd.DataFrame()

i=1
for col in data.columns:
    if i%4==1:
        total_cases[col]=data[col]
    if i%4==3:total_cured[col]=data[col]
    if i%4==0:total_deaths[col]=data[col]
    i+=1

total_cases.drop(index=[np.NaN], axis=0, inplace=True)
total_cured.drop(index=[np.NaN], axis=0, inplace=True)
total_deaths.drop(index=[np.NaN], axis=0, inplace=True)
total_cases = total_cases.astype(int)
total_cured = total_cured.astype(int)
total_deaths = total_deaths.astype(int)

In [5]:
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2020, 3, 10)
end_dt = date(2020, 7, 12)
time_axis = []
for dt in daterange(start_dt, end_dt):
    time_axis.append(dt)
date= pd.to_datetime(time_axis)

In [6]:
tot = total_cases.copy().transpose()
new_cases = tot.copy()
for row in range(1,len(tot.index)):
    new_cases.iloc[row,0:tot.shape[1]] = (tot.iloc[row,0:tot.shape[1]]-tot.iloc[row-1,0:tot.shape[1]]).astype(int)

new_cases['date'] = date

In [7]:
cases = new_cases.copy()
other_states = [col for col in cases.columns if col not in ["Delhi", "Maharashtra", "Tamil Nadu", "Totals"]]
cases["Rest of India"] = cases[other_states].sum(axis=1)
# cases
curr_pars = cases[["Delhi", "Maharashtra", "Tamil Nadu", "Rest of India", "Totals"]]
# curr_pars

In [8]:
# GENERATING DATA FOR TOTALS
cases = curr_pars[['Totals']]

In [9]:
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
# define contrived series
series = Series(cases['Totals'])
# print(series)
# prepare data for normalization
values = series.values
values = values.reshape((len(values), 1))
# train the normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(values)
print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
# normalize the dataset and print
normalized = scaler.transform(values)
# print(normalized)
# inverse transform and print
inversed = scaler.inverse_transform(normalized)
# print(inversed)

Min: 3.000000, Max: 29029.000000


In [10]:
count = 1
for date in new_cases['date']:
    if (str(date) == "2020-05-31 00:00:00"):
        break
    count += 1
data = np.array(normalized)
data = data.reshape((data.shape[0]))

In [11]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X),np.array(y)

In [12]:
# Generating train data upto 31st May 2020
train_data = data[:count]

# Remaining is test data
test_data  = data[count:]
print(test_data)
# choose a number of time steps
n_steps_in, n_steps_out = 5,5

# split into samples
train_X, train_Y = split_sequence(train_data, n_steps_in, n_steps_out)
train = np.concatenate((train_X, train_Y), axis=1)
train = pd.DataFrame(train, columns = ['Input1', 'Input2','Input3','Input4','Input5','Output1','Output2','Output3','Output4','Output5'])
print(train.head())

#summarize the data
for i in range(len(train_X)):
	print(train_X[i], train_Y[i])

# split into samples
test_X, test_Y= split_sequence(test_data, n_steps_in, n_steps_out)

test = np.concatenate((test_X, test_Y), axis=1)
test = pd.DataFrame(test, columns = ['Input1', 'Input2','Input3','Input4','Input5','Output1','Output2','Output3','Output4','Output5'])
print(test.head())

# summarize the data
for i in range(len(test_X)):
    print(test_X[i], test_Y[i])

train.to_csv('Processed_Data/train.csv')
test.to_csv('Processed_Data/test.csv')



[0.28422793 0.25439261 0.28240198 0.30803418 0.33490664 0.3204713
 0.32918762 0.37028871 0.31385654 0.32612141 0.3544753  0.39857369
 0.40604975 0.42975264 0.41214773 0.34286502 0.35767932 0.42885689
 0.46024254 0.48835527 0.53565769 0.55512299 0.50806174 0.54568318
 0.57076414 0.60852339 0.64249294 0.68955419 0.68924413 0.64721284
 0.64535244 0.6657824  0.74764005 0.81995452 0.86877282 0.82119479
 0.76665748 0.77923241 0.8838972  0.92079515 0.95969131 1.        ]
     Input1    Input2    Input3    Input4    Input5   Output1   Output2  \
0  0.000965  0.000345  0.000345  0.000138  0.000000  0.000792  0.000034   
1  0.000345  0.000345  0.000138  0.000000  0.000792  0.000034  0.000448   
2  0.000345  0.000138  0.000000  0.000792  0.000034  0.000448  0.000345   
3  0.000138  0.000000  0.000792  0.000034  0.000448  0.000345  0.000655   
4  0.000000  0.000792  0.000034  0.000448  0.000345  0.000655  0.001378   

    Output3   Output4   Output5  
0  0.000448  0.000345  0.000655  
1  0.000345 